Demo for running OORB propagation using ADAM. Still WIP.

In [ ]:
from adam import Service

from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
from adam import BatchPropagation
from adam import BatchPropagations
from adam import RunnableManager
import time
import os

In [ ]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager(os.getcwd() + '/config.json').get_config('dev')
auth_rest = AuthenticatingRestProxy(RestRequests(config.get_url()), config.get_token())

In [ ]:
service = Service(config)
service.setup()

In [ ]:
working_project = service.new_working_project()

In [ ]:
# 6x1 state vector (position [km], velocity [km/s]) for a perfectly circular orbit at about 1.5 AU
state_vec = [2.244e8, 0, 0,
             0, 24.31892597576429, 0]
# Period of this orbit is 671.0340959304685 days

In [ ]:
propagation_params = PropagationParams({
    'start_time': '2019-01-01T00:00:00Z',     # propagation start time ignored by OORB propagator
    'end_time':   '2037-05-16T08:10:58.884Z', # propagation end time in ISO format
    'project_uuid': config.get_workspace(),
    'executor': 'OORB',

#     'propagator_uuid': '00000000-0000-0000-0000-000000000002',  # force model
#     'description': 'some description'       # description of run
})

opm_params = OpmParams({
    'epoch': '2019-01-01T00:00:00Z',
    'state_vector': state_vec,
})

print(propagation_params)

In [ ]:
single = BatchPropagation(propagation_params, opm_params)
props = BatchPropagations(service.rest)

In [ ]:
RunnableManager(props, [single], working_project.get_uuid()).run()

In [ ]:
print(single.get_runnable_state().get_calc_state())
print(single.get_runnable_state().get_error())

For now, results from OORB are not parsed. They are available in the ephemeris field in their native form and units. Conversion is TBD.

In [ ]:
children = props.get_children(single.get_uuid())
result = children[0]
print(result.get_runnable_state().get_calc_state())
print(result.get_runnable_state().get_error())

In [ ]:
# This is raw OORB output for now.
print(result.get_ephemeris())

In [ ]:
#Cleans up working project and any batches and runs that were made within the project.
service.teardown()